Steps for EDA:
2. Summary of each nikaya (from Dhammatalks.org)
3. Number of suttas in each Nikaya (group by)
4. Number of words in each Nikaya
5. Number of pages
6. Average reading time
1. Representation of most linked from see also

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('./sutta_csv/cleaned/df_all_clean_2.csv')

In [3]:
df.shape

(1306, 9)

In [4]:
df.groupby('nikaya').count()

,title,ref,title_url,summary,text_full,intro,notes,see_also
nikaya,,,,,,,,
Anguttara Nikaya,385,385,385,385,385,385,385,385
Digha Nikaya,14,14,14,14,14,14,13,14
Khuddaka Nikaya,427,427,427,427,427,427,427,427
Majjhima Nikaya,101,101,101,101,101,101,101,101
Samyutta Nikaya,379,379,379,379,379,379,379,379


In [5]:
# Making word counts

df['word_count'] = df['text_full'].str.count(' ') + 1

mean_words = df.groupby('nikaya')[['word_count']].mean()

mean_words = mean_words.rename(columns = {
                                 'word_count': 'mean_words'})

In [6]:
#Total sum of words

sum_words = df.groupby('nikaya')[['word_count']].sum()

sum_words = sum_words.rename(columns = {
                                 'word_count': 'sum_words'})


In [7]:
sum_mean = pd.merge(sum_words, mean_words, on = 'nikaya')

In [8]:
# Assuming 250 words per page and 265 WPM reading speed

sum_mean['total_pages'] = sum_mean['sum_words'] / 250
sum_mean['average_pages'] = sum_mean['mean_words'] / 250
sum_mean['total_read_time_hrs'] = sum_mean['sum_words'] / (265 * 60)
sum_mean['average_read_time_mins'] = sum_mean['mean_words'] / 265

In [9]:
sum_mean

,sum_words,mean_words,total_pages,average_pages,total_read_time_hrs,average_read_time_mins
nikaya,,,,,,
Anguttara Nikaya,296039,768.932468,1184.156,3.075730,18.618805,2.901632
Digha Nikaya,141063,10075.928571,564.252,40.303714,8.871887,38.022372
Khuddaka Nikaya,415757,973.669789,1663.028,3.894679,26.148239,3.674226
Majjhima Nikaya,319603,3164.386139,1278.412,12.657545,20.100818,11.941080
Samyutta Nikaya,265004,699.218997,1060.016,2.796876,16.666918,2.638562


In [10]:
full_canon = sum_mean.sum()
full_canon.name = 'Full Canon'
sum_mean = sum_mean.append(full_canon.T)

In [11]:
sum_mean.reset_index(inplace = True)

# DataFrame with relevant information for each nikaya

#### Summary of each Nikaya

In [12]:
sutta_names = {'Majjhima Nikaya': [101], 'Samyutta Nikaya': [379], 'Digha Nikaya': [14], 'Anguttara Nikaya': [385], 'Khuddaka Nikaya': [427], 'Full Canon': [1306]}

In [13]:
df_eda = pd.DataFrame(sutta_names).T

df_eda.reset_index(inplace=True)

df_eda = df_eda.rename(columns = {0: "count",
                                 'index': 'nikaya'})

In [14]:
df_eda = pd.merge(df_eda, sum_mean, on = 'nikaya')

In [15]:
df_eda

,nikaya,count,sum_words,mean_words,total_pages,average_pages,total_read_time_hrs,average_read_time_mins
0,Majjhima Nikaya,101,319603.0,3164.386139,1278.412,12.657545,20.100818,11.941080
1,Samyutta Nikaya,379,265004.0,699.218997,1060.016,2.796876,16.666918,2.638562
2,Digha Nikaya,14,141063.0,10075.928571,564.252,40.303714,8.871887,38.022372
3,Anguttara Nikaya,385,296039.0,768.932468,1184.156,3.075730,18.618805,2.901632
4,Khuddaka Nikaya,427,415757.0,973.669789,1663.028,3.894679,26.148239,3.674226
5,Full Canon,1306,1437466.0,15682.135964,5749.864,62.728544,90.406667,59.177872


In [16]:
# Getting means where sums are currently

df_eda.at[5,'mean_words'] = df_eda.at[5,'mean_words'] / 5
df_eda.at[5,'average_pages'] = df_eda.at[5,'average_pages'] / 5
df_eda.at[5,'average_read_time_mins'] = df_eda.at[5,'average_read_time_mins'] / 5

In [17]:
df_eda

,nikaya,count,sum_words,mean_words,total_pages,average_pages,total_read_time_hrs,average_read_time_mins
0,Majjhima Nikaya,101,319603.0,3164.386139,1278.412,12.657545,20.100818,11.941080
1,Samyutta Nikaya,379,265004.0,699.218997,1060.016,2.796876,16.666918,2.638562
2,Digha Nikaya,14,141063.0,10075.928571,564.252,40.303714,8.871887,38.022372
3,Anguttara Nikaya,385,296039.0,768.932468,1184.156,3.075730,18.618805,2.901632
4,Khuddaka Nikaya,427,415757.0,973.669789,1663.028,3.894679,26.148239,3.674226
5,Full Canon,1306,1437466.0,3136.427193,5749.864,12.545709,90.406667,11.835574


In [18]:
# df_eda.to_csv('./sutta_csv/eda/df_eda.csv', index = False)

# DataFrame with See also recommendation counts

#### Drop all 'No Additional'

In [19]:
df = df[~df.see_also.str.contains("No Additional")]

#### Removing first colon and \xa

In [20]:
df['see_also_cl'] = df['see_also'].str.replace(': ', '').copy()

In [21]:
df['see_also_cl'] = df['see_also_cl'].str.replace('\xa0', ' ').copy()

In [22]:
#df['see_also_cl'] = df['see_also_cl'].str.replace('\ ', ' ').copy()

#### Split at each semicolon

In [23]:
df['see_also_cl'] = df['see_also_cl'].str.split(';')

#### Put all in one list, stripped

In [24]:
all_see = []
for i in df['see_also_cl']:
    for j in i:
        all_see.append(j.strip())

#### List for each Sutta Ref

In [25]:
MN = []
SN = []
DN = []
AN = []
KN = []

for i in all_see:
    if i.__contains__('MN'):
        MN.append(i)
    elif i.__contains__('SN'):
        SN.append(i)
    elif i.__contains__('DN'):
        DN.append(i)
    elif i.__contains__('AN'):
        AN.append(i)
    else: 
        KN.append(i)

#### Function for turn dictionary to dataframe

In [26]:
def count_df(sutta_list):
    sutta_counts = {x: [sutta_list.count(x)] for x in sutta_list}
    df_sc = pd.DataFrame(sutta_counts).T
    df_sc.reset_index(inplace=True)
    df_sc = df_sc.rename(columns = {"index": "sutta"})
    
    return df_sc

#### Making and concatenating all dataframes

In [27]:
df_mn = count_df(MN)
df_mn = df_mn.rename(columns = {0: "count"})
df_mn.sort_values(by = 'count', inplace = True, ascending= False)
df_mn['ref'] = 'MN'

df_sn = count_df(SN)
df_sn = df_sn.rename(columns = {0: "count"})
df_sn.sort_values(by = 'count', inplace = True, ascending= False)
df_sn['ref'] = 'SN'

df_dn = count_df(DN)
df_dn = df_dn.rename(columns = {0: "count"})
df_dn.sort_values(by = 'count', inplace = True, ascending= False)
df_dn['ref'] = 'DN'

df_an = count_df(AN)
df_an = df_an.rename(columns = {0: "count"})
df_an.sort_values(by = 'count', inplace = True, ascending= False)
df_an['ref'] = 'AN'

df_kn = count_df(KN)
df_kn = df_kn.rename(columns = {0: "count"})
df_kn.sort_values(by = 'count', inplace = True, ascending= False)
df_kn['ref'] = 'KN'

In [28]:
df_all_count = pd.concat([df_mn, df_sn, df_dn, df_an, df_kn], axis=0)

In [29]:
#df_all_count.to_csv('./sutta_csv/eda/df_all_counts.csv', index = False)

# Countvectorizer Set Up

In [30]:
df_2 = df[:3].copy()

In [31]:
my_stopwords2 = list(CountVectorizer(stop_words = 'english').get_stop_words()) + ['just', 
                                                                                  'like', 've', 'don','really', 'things', 'deleted', 'feel', 'want',
                                                                                  'years', 'help', 'going', 'make', 'day', 'having', 'bad', 'got', 'way', 
                                                                                  "I'm", "I've", "I'll", 'n', 't', 's', 'm', 'g', 'll', 'doesn']

In [32]:
cvec = CountVectorizer(stop_words = my_stopwords2)

cvec.fit(df_2['text_full'])

z = cvec.transform(df_2['text_full'])
z_df = pd.DataFrame(z.todense(), columns = cvec.get_feature_names())

In [33]:
z_df['ref'] = df_2['ref']

In [34]:
z_df['nikaya'] = df_2['nikaya']

In [35]:
z_df

,abandoned,abandoning,abandons,abiding,accompanied,accordance,actions,activities,addressed,afflictions,aging,alert,all6,alms,analysis,animal,anāthapiṇḍika,appearance,appeared,appearing,appropriate,appropriately,arahant,ardent,arise,arisen,arises,arose,aroused,arrival,aspirations,associating,assurance,attained,attend,attending,attends,attention,attitudes,aversion,avoid,avoiding,avoids,awakened,awakening,awareness,away,awe,beautification,beautiful,beings,belonged,benefit,bird,birth,black,blameless,blessed,bodhisatta,bodily,body,bondage,born,bound,brahman,brahmans,brahmā,bramble,break,breakup,bright,bulk,bull,burden,called,calm,came,carry,case,cause,cessation,cesspool,change,chasm,clan,clear,cliff,close,cognized,cognized4,cold,come,comfort,coming,companions,compelling,completion,comprehend,comprehended,conceit,concentrated,concentration,conduct,conqueror,conqueror2,consciousness,constant,consummate,contemplative,contemplatives,continuance,contortion,contraction,conviction,cosmic,counteract,courteous,courtesies,covering,covetous,craving,create,curing,dark,darkness,death,declare,defilement,delight,delighted,delights,delusion,dependent,deprivation,designated,desirous,despair,destination,destroy,destroyed,destroying,destroys,destructive,details,devas,develop,developing,develops,devoid,dhamma,did,dimension,directed,directly,disagreeable,disappearance,discern,discerned,discernment,discerns,disciple,disciplined,disease,disparage,dispassion,dispels,displeased,displeasing,distress,divine,does,dog,doubt,doubting,dread,drooling,drop,drowsiness,dwell,dwelling,dwellings,dwells,ear,earlier,earth,easy,effluent,effluents,eighth,elation,elephant,end,ended,ending,endowed,endure,endures,enjoy,enjoyment,entered,eons,equanimity,equanimous,established,eternal,eternity,evaluation,evaluations,everlasting,evil,example,exchange,exchanged,existence,expansion,experience,eye,eyes,factor,faculty,fading,fallen,fame,families,fault,fear,feeling,feelings,felt,fetter,fetters,fever,fiercely,fifteenth,finished,fit,flies,follower,food,forest,forests,forms,forth,fortnight,fortunate,forward,fourteenth,fourth,free,freed,freedom,friendly,friends,fruit,fulfilled,future,gains,generations,goal,gone,good,goodwill,gotama,grasping,gratified,greetings,grove,habitats,habits,hair,happens,happiness,harmfulness,heard,heart,heat,heavenly,heedful,held,hell,helper,hidden,holy,home,homelessness,horripilation,horse,human,hunger,ideas,identification,idiocy,idiot,idiots,ignorance,ill,illness,immediate,imperturbability,inappropriate,inappropriately,inclemencies,increase,increases,inferior,infinitude,influence,inspirer,inspiring,instead,instructed,integrity,intellect,internal,intoxication,inwardly,isolated,jeta,jhāna,jāṇussoṇin,kamma,kinds,know,knower,knowing,knowledge,known,knows,lacking,laid,lamentation,lamp,later,lay,lazy,leader,leading,leaves,let,letting,lie,life,light,lines,listen,live,livelihood,lives,lodging,long,lord,lost,lower,lunar,lying,magnificent,maintain,malleable,manifold,master,maximum,means,medicinal,menacing,mental,mind,mindful,mindfulness,minds,mn,modes,modest,monastery,monk,monks,mosquitoes,muddled,multiplicity,multiplicity5,near,new,night,nights,noble,non,nose,nothingness,observant,occasion,occur,occurred,offerings,old,ones,open,origination,overcome,overeating,overturned,pain,painful,pains,pajāpati,panic,park,parts,passing,passion,passionate,past,patch,pay,penetration,people,perceive,perceives,perceiving,perception,perception3,perplexed,persistence,person,phenomena,piercing,place,places,plane,playfully,pleasant,pleasure,pleasures,pliant,plunder,practices,praise,precisely,present,protection,purified,purity,purpose,putting,qualities,quite,racking,radiant,rapture,realm,reappearance,reappeared,reasoning,reasons,recognized,recollected,recollecting,reflecting,refuge,regard,release,released,relief,relinquishment,remained,remember,reptiles,requisites,resolute,resort,resorts,responded,restless,restrained,restraining,restraint,resulting,reveal,revile,reviled,rid,right,rightly,rip

In [36]:
# z_df = z_df.T

In [37]:
# z_df.reset_index(inplace=True)

In [38]:
z_df = z_df.rename(columns = {"index": "words",
                             0: 'MN 1',
                              1: 'MN 2',
                             2: 'MN 4'})


In [39]:
# z_df.reset_index(inplace = True)

In [40]:
z_df.to_csv('./sutta_csv/cleaned/Tableau/countv.csv', index = False)